In [ ]:
import LinearAlgebra

In [ ]:
import Random

In [ ]:
import BenchmarkTools

# test scoring function
formulate as a minmization problem

In [ ]:
function Score(Answer)
    return(ceil(Int64, (LinearAlgebra.norm(Answer .- 500))^2))
end

In [ ]:
function TestAlgorithm(dims = 3, nparticles = 10, maxiter = 500)
    # generate the starting points for the algorithm
    # store these in a matrix, where each column is a particle, and each row is a dimension in the search space
    particles = rand(1:1000, dims, nparticles)
    velocities = rand(-3:3, dims, nparticles)
    direction = zeros(Int64, dims, nparticles)
    scores = zeros(Int64, nparticles)
    
    # create a set to store the points that we have visited, to avoid looking at the same points repeatedly
    visited = Set{Vector{Int64}}()
    
    # give a size hint for the set of visited points, hopefully preventing it from growing repeatedly
    sizehint!(visited, nparticles * maxiter)
    
    # save all the locations we've visited
    mapslices(x -> push!(visited, x), particles, dims = 1)
    
    iter = 1
    
    best_particle = zeros(Int64, dims)
    global_optimum = Inf
    
    # loop until convergence
    # while iter < maxiter
    while iter < maxiter
        # compute the current scores of each particle
        map!(x -> Score(x), scores, collect(eachcol(particles)))
        
        # find the current best particle
        best_score = findmin(scores)
        
        # if this score is better than the current best score, update
        if best_score[1] < global_optimum[1]
            global_optimum = best_score[1]
            best_particle .= particles[:,best_score[2]]
        end
        
        # figure out which direction each particle needs to go to head towards this best particle
        # need to work on this component
        # and weighting the amount that the velocities change
        # also need to take advantage of the fact that we can store the visited vectors as we go!
        direction .= mapslices(x -> sign.(best_particle .- x), particles, dims = 1)

        # add this change in direction to the current velocities vector
        velocities += direction

        # update the particle positions based on the velocities
        particles += velocities
        
        # save the new locations we've visited
        mapslices(x -> push!(visited, x), particles, dims = 1)

        iter += 1
    end    
    return(global_optimum, best_particle)
end

In [ ]:
Random.seed!(100)
a = TestAlgorithm(100, 1000, 500)

In [ ]:
BenchmarkTools.@btime(TestAlgorithm(100, 1000, 500))